In [1]:
from jupyterUtils import *
%load_ext autoreload
%autoreload 2
%matplotlib inline

print("\n\n")
print("__________________________________________________________________________________________")
!nvidia-smi
print("__________________________________________________________________________________________")
print("\n\n")

def exeIt2(cmd):
    releaseTf()
    print("____________________________________________________________________________________________________")
    cmd = decodeCmd(cmd, sepBy=' ')
    print("\n\n\n", cmd, "\n\n\n")
    print("____________________________________________________________________________________________________")
    !{cmd}
    r, c, d = 0, '', ''




__________________________________________________________________________________________
Sun Jan 17 23:27:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 166...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8     6W /  N/A |    392MiB /  5944MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+------------------

In [ ]:
def gitPull(repoPath, force):
    if force:
        # !cd {repoPath};git checkout master
        print("_____________________________________________________________")
        !cd {repoPath};git fetch --all
        !cd {repoPath};git reset --hard origin/master
        print("_____________________________________________________________")
    else:
        print("_____________________________________________________________")
        !cd {repoPath};git pull
        print("_____________________________________________________________")
    reloadPy()
    


def gitPush(repoPath, commitMsg, push, clear, force):
    !cd {repoPath};git config --global user.email 'vishnu@hippovideo.io'
    !cd {repoPath};git config --global user.name 'vishnu'
    if clear:
        !cd {repoPath};git rm -r --cached .
    !cd {repoPath};git add . --all
    !cd {repoPath};git status
    fixPrint = '{print $2}'
    print("_____________________________________________________________")
    !cd {repoPath};echo $repoPath;git status --porcelain | awk '{fixPrint}' | xargs ls -hs | sort -h;
    print("_____________________________________________________________")
    if push and not clear:
        print("_____________________________________________________________")
        !cd {repoPath};git commit -a -m '{commitMsg}'  # commit in Colab
        if force:
            !cd {repoPath};git push origin master --force         # push to github
        else:
            !cd {repoPath};git push origin master     # push to github
        print("_____________________________________________________________")

In [ ]:
def downloadDB(url, des, rename='', noCertificate=False):
    url = [c.strip() for c in url.replace('\n', ';').split(';')]
    url = ' '.join([c for c in url if c and not c.startswith('#')])
    dirop(des)
    downloadCmd = f'cd "{des}";'
    if not exists(url):
        print(url)
        print('\n\n\n\n')
        if 'https://github.com' in url:
            downloadCmd += f'git clone "{url}";' if  '.' not in basename(url) else f'wget {url};'
        elif 'https://drive.google.com' in url:
            if '/d/' in url:
                gid = url
                gid = gid.split('/d/')[1]
                gid = gid.split('/')[0]
            elif 'id=' in url:
                gid = url
                gid = gid.split('id=')[1]
                gid = gid.split('&')[0]
            downloadCmd += f'gdown https://drive.google.com/uc?id={gid};'
        elif 'https://www.youtube.com' in url:
            downloadCmd += f"youtube-dl '{url}' --print-json --restrict-filenames -o '%(id)s.%(ext)s'"
        elif noCertificate:
            downloadCmd += f'wget --no-check-certificate "{url}";'
        else:
            downloadCmd += f'wget "{url}";'
        old = set(glob(f'{des}/*'))
        print("____________________________________________________________________________________________________________________")
        print(f"\n             {downloadCmd}\n")
        print("____________________________________________________________________________________________________________________")
        !{downloadCmd}
        returnData = list(set(glob(f'{des}/*')) - old)
    else:
        returnData = [url]
    if len(returnData) != 1:
        print("skipping unzip no. file downloaded != 1")
        print("returnData:", returnData)
        return returnData
    returnData = returnData[0]
    if rename:
        returnData = dirop(returnData, mv=join(dirname(returnData), rename))
    try:
        print("unzip dataset")
        returnData = unzipIt(returnData, returnData.split('.')[0])
        print("zip: ", returnData)
        print("des: ", returnData.split('.')[0])
    except:
        print("skipping unzipp")
    return returnData



def downloadVideo(vpath, compress):
    if type(vpath) == str:
        if compress:
            des = dirop(f"/tmp/{basename(vpath)}")
            !ffmpeg -i {vpath} -vcodec libx265 -crf 28 {des} -y
        else:
            des = vpath
    else:
        if compress:
            des = dirop(f"/tmp/{getTimeStamp()}")
            for v in vpath:
                print("compressing", v)
                summaryPath = f"{des}/{basename(v)}"
                !ffmpeg -i {v} -vcodec libx265 -crf 28 {summaryPath} -y
        else:
            des = vpath
        des = zipIt(des, f'/tmp/{basename(des)}.zip')
    print("des", des)
    coFile.download(des)
    return des